In [1]:
%matplotlib inline

In [5]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


## Import data

In [6]:
from keras.datasets import mnist

In [7]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

15228928/15296311 [============================>.] - ETA: 0s

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [8]:
X_test = np.expand_dims(X_test, 1)
X_train = np.expand_dims(X_train, 1)

In [9]:
X_train.shape

(60000, 1, 28, 28)

In [10]:
y_train[:6]

array([5, 0, 4, 1, 9, 2], dtype=uint8)

In [11]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [12]:
y_train[:6]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [14]:
mean_x = X_train.mean().astype(np.float32)
std_x = X_train.std().astype(np.float32)

In [15]:
def norm_input(x):
    return (x - mean_x) / std_x

## Linear model

create linear model

In [20]:
def generate_linear_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
lm = generate_linear_model()
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 1, 28, 28)     0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 784)           0           lambda_3[0][0]                   
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 10)            7850        flatten_3[0][0]                  
Total params: 7850
____________________________________________________________________________________________________


create batches

In [28]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size = 64)
test_batches = gen.flow(X_test, y_test, batch_size = 64)

fit model with default learning rate

try lm.fit() to use validation_split

In [29]:
lm.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches, nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.4363 - acc: 0.8706 - val_loss: 0.2987 - val_acc: 0.9128


change learning rate and refit

In [30]:
lm.optimizer.lr = 0.1
lm.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches, nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.2994 - acc: 0.9142 - val_loss: 0.2820 - val_acc: 0.9186


In [25]:
lm.optimizer.lr = 0.01
lm.fit_generator(batches, batches.N, nb_epoch = 3, validation_data = test_batches, nb_val_samples = test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 11s - loss: 0.2572 - acc: 0.9285 - val_loss: 0.2645 - val_acc: 0.9275
Epoch 2/3
60000/60000 [==============================] - 11s - loss: 0.2580 - acc: 0.9269 - val_loss: 0.2897 - val_acc: 0.9207
Epoch 3/3
60000/60000 [==============================] - 11s - loss: 0.2572 - acc: 0.9282 - val_loss: 0.2928 - val_acc: 0.9198


## Single dense layer

In [45]:
def generate_fc_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [46]:
fc_model = generate_fc_model()
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 1, 28, 28)     0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
flatten_6 (Flatten)              (None, 784)           0           lambda_6[0][0]                   
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 512)           401920      flatten_6[0][0]                  
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 10)            5130        dense_8[0][0]                    
Total params: 407050
______________________________________________________________________

In [47]:
fc_model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)
fc_model.summary()

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.1947 - acc: 0.9404 - val_loss: 0.1045 - val_acc: 0.9662
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 1, 28, 28)     0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
flatten_6 (Flatten)              (None, 784)           0           lambda_6[0][0]                   
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 512)           401920      flatten_6[0][0]                  
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 10)            5130      

In [48]:
fc_model.optimizer.lr = 0.1
fc_model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.0813 - acc: 0.9745 - val_loss: 0.0882 - val_acc: 0.9720


In [49]:
fc_model.optimizer.lr = 0.01
fc_model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.0519 - acc: 0.9839 - val_loss: 0.0667 - val_acc: 0.9810


## Basic VGG-like CNN

?? understand Convolution2D

In [70]:
def generate_simply_vgg_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            Convolution2D(32,3,3, activation='relu'),
            Convolution2D(32,3,3, activation='relu'),
            MaxPooling2D(),
            Convolution2D(64,3,3, activation='relu'),
            Convolution2D(64,3,3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            Dense(512, activation='relu'),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [71]:
model = generate_simply_vgg_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_11 (Lambda)               (None, 1, 28, 28)     0           lambda_input_11[0][0]            
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 32, 26, 26)    320         lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution2d_18 (Convolution2D) (None, 32, 24, 24)    9248        convolution2d_17[0][0]           
____________________________________________________________________________________________________
maxpooling2d_9 (MaxPooling2D)    (None, 32, 12, 12)    0           convolution2d_18[0][0]           
___________________________________________________________________________________________

In [72]:
model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 20s - loss: 0.1962 - acc: 0.9378 - val_loss: 0.0884 - val_acc: 0.9708


In [73]:
model.optimizer.lr = 0.1
model.fit_generator(batches, batches.N, nb_epoch = 3, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 19s - loss: 0.0716 - acc: 0.9778 - val_loss: 0.0457 - val_acc: 0.9854
Epoch 2/3
60000/60000 [==============================] - 20s - loss: 0.0541 - acc: 0.9835 - val_loss: 0.0422 - val_acc: 0.9856
Epoch 3/3
60000/60000 [==============================] - 19s - loss: 0.0450 - acc: 0.9864 - val_loss: 0.0515 - val_acc: 0.9839


In [74]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 19s - loss: 0.0431 - acc: 0.9870 - val_loss: 0.0369 - val_acc: 0.9889
Epoch 2/2
60000/60000 [==============================] - 19s - loss: 0.0392 - acc: 0.9876 - val_loss: 0.0356 - val_acc: 0.9877


## Data augmentation

In [75]:
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train, y_train, batch_size = 64)
test_batches = gen.flow(X_test, y_test, batch_size = 64)

In [76]:
model = generate_simply_vgg_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_12 (Lambda)               (None, 1, 28, 28)     0           lambda_input_12[0][0]            
____________________________________________________________________________________________________
convolution2d_21 (Convolution2D) (None, 32, 26, 26)    320         lambda_12[0][0]                  
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (None, 32, 24, 24)    9248        convolution2d_21[0][0]           
____________________________________________________________________________________________________
maxpooling2d_11 (MaxPooling2D)   (None, 32, 12, 12)    0           convolution2d_22[0][0]           
___________________________________________________________________________________________

In [77]:
model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.1982 - acc: 0.9360 - val_loss: 0.0746 - val_acc: 0.9756


In [78]:
model.optimizer.lr = 0.1
model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 0.0687 - acc: 0.9787 - val_loss: 0.0681 - val_acc: 0.9783


In [79]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.N, nb_epoch = 3, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 19s - loss: 0.0518 - acc: 0.9835 - val_loss: 0.0401 - val_acc: 0.9878
Epoch 2/3
60000/60000 [==============================] - 19s - loss: 0.0443 - acc: 0.9869 - val_loss: 0.0299 - val_acc: 0.9898
Epoch 3/3
60000/60000 [==============================] - 19s - loss: 0.0410 - acc: 0.9878 - val_loss: 0.0439 - val_acc: 0.9859


In [80]:
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 19s - loss: 0.0371 - acc: 0.9883 - val_loss: 0.0325 - val_acc: 0.9903
Epoch 2/2
60000/60000 [==============================] - 19s - loss: 0.0350 - acc: 0.9890 - val_loss: 0.0330 - val_acc: 0.9894


## Batch normalization and data augmentation

In [81]:
def generate_bn_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(32,3,3, activation='relu'),
            MaxPooling2D(),
            BatchNormalization(axis=1),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(64,3,3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            BatchNormalization(axis=1),
            Dense(512, activation='relu'),
            BatchNormalization(axis=1),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [82]:
model = generate_bn_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_13 (Lambda)               (None, 1, 28, 28)     0           lambda_input_13[0][0]            
____________________________________________________________________________________________________
convolution2d_25 (Convolution2D) (None, 32, 26, 26)    320         lambda_13[0][0]                  
____________________________________________________________________________________________________
batchnormalization_6 (BatchNormal(None, 32, 26, 26)    64          convolution2d_25[0][0]           
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 32, 24, 24)    9248        batchnormalization_6[0][0]       
___________________________________________________________________________________________

In [88]:
model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 45s - loss: 0.0402 - acc: 0.9874 - val_loss: 0.0388 - val_acc: 0.9884


In [89]:
model.optimizer.lr = 0.1
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 45s - loss: 0.0360 - acc: 0.9890 - val_loss: 0.0291 - val_acc: 0.9902
Epoch 2/2
60000/60000 [==============================] - 45s - loss: 0.0351 - acc: 0.9887 - val_loss: 0.0270 - val_acc: 0.9911


In [91]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 45s - loss: 0.0306 - acc: 0.9902 - val_loss: 0.0248 - val_acc: 0.9911
Epoch 2/2
60000/60000 [==============================] - 45s - loss: 0.0266 - acc: 0.9914 - val_loss: 0.0304 - val_acc: 0.9908


## Batchnorm + dropout + data augmentation

In [92]:
def generate_bn_do_model():
    model = Sequential([
            Lambda(norm_input, input_shape=(1,28,28)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(32,3,3, activation='relu'),
            MaxPooling2D(),
            BatchNormalization(axis=1),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(64,3,3, activation='relu'),
            MaxPooling2D(),
            Flatten(),
            BatchNormalization(axis=1),
            Dense(512, activation='relu'),
            BatchNormalization(axis=1),
            Dropout(0.5),
            Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [93]:
model = generate_bn_do_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_14 (Lambda)               (None, 1, 28, 28)     0           lambda_input_14[0][0]            
____________________________________________________________________________________________________
convolution2d_29 (Convolution2D) (None, 32, 26, 26)    320         lambda_14[0][0]                  
____________________________________________________________________________________________________
batchnormalization_11 (BatchNorma(None, 32, 26, 26)    64          convolution2d_29[0][0]           
____________________________________________________________________________________________________
convolution2d_30 (Convolution2D) (None, 32, 24, 24)    9248        batchnormalization_11[0][0]      
___________________________________________________________________________________________

In [94]:
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 45s - loss: 0.2314 - acc: 0.9301 - val_loss: 0.0707 - val_acc: 0.9777
Epoch 2/2
60000/60000 [==============================] - 45s - loss: 0.0930 - acc: 0.9709 - val_loss: 0.0468 - val_acc: 0.9840


In [95]:
model.optimizer.lr = 0.1
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 46s - loss: 0.0793 - acc: 0.9759 - val_loss: 0.0496 - val_acc: 0.9847
Epoch 2/2
60000/60000 [==============================] - 45s - loss: 0.0682 - acc: 0.9794 - val_loss: 0.0509 - val_acc: 0.9828


In [96]:
model.optimizer.lr = 0.01
model.fit_generator(batches, batches.N, nb_epoch = 2, validation_data = test_batches,
                       nb_val_samples = test_batches.N)

Epoch 1/2
60000/60000 [==============================] - 45s - loss: 0.0602 - acc: 0.9808 - val_loss: 0.0569 - val_acc: 0.9816
Epoch 2/2
60000/60000 [==============================] - 45s - loss: 0.0567 - acc: 0.9829 - val_loss: 0.0371 - val_acc: 0.9872


## Combine all together

In [ ]:
def fit_model():
    model = generate_bn_do_model()
    model.fit_generator(batches, batches.N, nb_epoch = 1, validation_data = test_batches,
                       nb_val_samples = test_batches.N)
    model.optimizer.lr = 0.1
    model.fit_generator(batches, batches.N, nb_epoch = 4, validation_data = test_batches,
                       nb_val_samples = test_batches.N)
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.N, nb_epoch = 8, validation_data = test_batches,
                       nb_val_samples = test_batches.N)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.N, nb_epoch = 12, validation_data = test_batches,
                       nb_val_samples = test_batches.N)
    return model

In [ ]:
models = [fit_model() for i in range(6)]

In [ ]:
model_path = 'models/'

In [ ]:
for i, m in enumerate(models):
    m.save_weights(model_path + 'mnist-cnn-' + str(i) + '.pk1')

In [ ]:
evals = np.array([m.evaluate(X_test, y_test, batch_size=256) for m in models])

In [ ]:
evals.shape
evals.mean(axis=0)

In [ ]:
all_preds = np.stack([m.predict(X_test, batch_size=256) for m in models])

In [ ]:
all_preds.shape

In [ ]:
avg_preds = all_preds.mean(axis=0)

In [ ]:
keras.metrics.categorical_accuracy(y_test, avg_preds).eval()